In [13]:
from youtube_transcript_api import YouTubeTranscriptApi
import tiktoken
from openai import OpenAI
import os
import json5
import json
import ast

In [14]:
video_id = "y3yAVZk3tyA"
ytt_api = YouTubeTranscriptApi()
fetched_transcript = ytt_api.fetch(video_id)

In [15]:
from googleapiclient.discovery import build
api_key = os.environ.get("YOUTUBE_API_KEY")
if not api_key:
    print("was not ablt to get the key")
video_id = "y3yAVZk3tyA"
youtube = build("youtube", "v3", developerKey=api_key)
request = youtube.videos().list(part="snippet", id=video_id)
response = request.execute()

video_title = response["items"][0]["snippet"]["title"]
print("Video title:", video_title)

Video title: Keyu Jin: China's Economy, Tariffs, Trade, Trump, Communism & Capitalism | Lex Fridman Podcast #477


In [16]:
fetched_transcript

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='- The following is a\nconversation with Keyu Jin,', start=0.18, duration=3.27), FetchedTranscriptSnippet(text='an economist at the London\nSchool of Economics,', start=3.45, duration=2.58), FetchedTranscriptSnippet(text="specializing in China's economy,", start=6.03, duration=2.4), FetchedTranscriptSnippet(text='international macroeconomics,\nglobal trade imbalances,', start=8.43, duration=3.12), FetchedTranscriptSnippet(text='and financial policy.', start=11.55, duration=1.89), FetchedTranscriptSnippet(text='She wrote the highly\nlauded book on China titled', start=13.44, duration=4.147), FetchedTranscriptSnippet(text='"The New China Playbook: Beyond\nSocialism and Capitalism,"', start=17.587, duration=4.943), FetchedTranscriptSnippet(text="that details China's\neconomic transformation", start=22.53, duration=2.97), FetchedTranscriptSnippet(text='since 1978 to today.', start=25.5, duration=3.12), FetchedTranscriptSnippet(text=

In [17]:
type(fetched_transcript)

youtube_transcript_api._transcripts.FetchedTranscript

In [18]:
len(fetched_transcript.snippets)

2348

In [19]:
fetched_transcript.video_id

'y3yAVZk3tyA'

In [20]:
type(fetched_transcript.snippets[0])

youtube_transcript_api._transcripts.FetchedTranscriptSnippet

In [21]:
print(fetched_transcript.snippets[0])

FetchedTranscriptSnippet(text='- The following is a\nconversation with Keyu Jin,', start=0.18, duration=3.27)


In [22]:
fetched_transcript.snippets[0].text

'- The following is a\nconversation with Keyu Jin,'

In [23]:
from youtube_transcript_api.formatters import JSONFormatter
formatter = JSONFormatter()
json_string_transcript = formatter.format_transcript(fetched_transcript)

In [24]:
json_transcript = json.loads(json_string_transcript)

In [25]:
type(json_transcript)

list

In [26]:
import json
print(json.dumps(json_transcript, indent=4))

[
    {
        "text": "- The following is a\nconversation with Keyu Jin,",
        "start": 0.18,
        "duration": 3.27
    },
    {
        "text": "an economist at the London\nSchool of Economics,",
        "start": 3.45,
        "duration": 2.58
    },
    {
        "text": "specializing in China's economy,",
        "start": 6.03,
        "duration": 2.4
    },
    {
        "text": "international macroeconomics,\nglobal trade imbalances,",
        "start": 8.43,
        "duration": 3.12
    },
    {
        "text": "and financial policy.",
        "start": 11.55,
        "duration": 1.89
    },
    {
        "text": "She wrote the highly\nlauded book on China titled",
        "start": 13.44,
        "duration": 4.147
    },
    {
        "text": "\"The New China Playbook: Beyond\nSocialism and Capitalism,\"",
        "start": 17.587,
        "duration": 4.943
    },
    {
        "text": "that details China's\neconomic transformation",
        "start": 22.53,
        "duratio

In [27]:
print(json_transcript[0]['text'])

- The following is a
conversation with Keyu Jin,


In [28]:
json_transcript[1]['text']

'an economist at the London\nSchool of Economics,'

In [29]:
print(json_transcript[0]['text'] + json_transcript[1]['text'])

- The following is a
conversation with Keyu Jin,an economist at the London
School of Economics,


In [30]:
len(json_transcript[0]['text'])

48

In [31]:
len(json_transcript[1]['text'])

47

In [32]:
len(json_transcript[0]['text'] + json_transcript[1]['text'])

95

In [33]:
len(json_transcript)

2348

This code is to calcualte the len of all the text we have so i can verify that text_chunks has all the text that was in the transcript

In [34]:
text = ""
for json_tr in json_transcript:
    text = text + json_tr['text']
print(len(text))

91611


In [35]:
encoding = tiktoken.encoding_for_model("gpt-4.1-mini")
tokens = encoding.encode("Your text here")
token_count = len(tokens)
print(token_count)


3


In [36]:
MAX_TOKENS = 1500
text_chunks = []
chunk_len = 0
text = ""
for json_tr in json_transcript:
    current_chunk_len = len(encoding.encode(json_tr['text']))
    # print(f"current_chunk_len: {current_chunk_len}")
    # print(f"text: {tr['text']}")
    if chunk_len + current_chunk_len <= MAX_TOKENS:
        text = text + " " +  json_tr['text']
        chunk_len += current_chunk_len
        # print(f"new text : {text}")
        # print(f"")
    else:
        # break
        text_chunks.append(text)
        text = json_tr['text']
        chunk_len = current_chunk_len
if text:
    text_chunks.append(text)

In [37]:
len(text_chunks)

14

In [38]:
total_number_chars = 0
for tch in text_chunks:
    total_number_chars += len(tch)
print(total_number_chars)

93946


93946 > 91611 the differnce is becausei have added spaces when creating text_chunks

In [39]:
len(text_chunks)


14

In [40]:
token = os.environ.get("GITHUB_TOKEN")   # .get() avoids KeyError if missing

if not token:
        raise RuntimeError("⚠️ GITHUB_TOKEN is not set. Did you run `direnv allow`?")

In [41]:
client = OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=token,
)

In [42]:
def llm(prompt):
    response = client.chat.completions.create(
        messages = [{"role": "user", "content":prompt,}],
        model = "openai/gpt-4o-mini",
        top_p = 1
    )
    return response.choices[0].message.content

In [43]:
response = llm("What is the capital of morroco")

In [44]:
response

'The capital of Morocco is Rabat.'

In [50]:
llm_chunking_prompt_template = """ 
You are a helpful assistant that organizes transcript text into **logical, semantically coherent chunks**.

I will provide you with text chunks from a video transcript. Each chunk may be long or short and may contain multiple sentences.

Your task:
1. Split and organize the input into **logical sections** where each section:
   - Covers a coherent topic or idea.
   - Is not too long (aim for ~200–500 words per section, adjust naturally based on content).
2. Preserve the original text as much as possible (do not rewrite or summarize unnecessarily).
3. For each chunk, also generate:
   - **title**: a concise, descriptive title summarizing the main idea of the chunk. Use the most important keywords from the text.
   - **keywords**: a list of 5–10 important keywords or phrases that capture the main concepts in the chunk.

4. Output the sections as **JSON**, in this format:

[
  {{
    "video_id" : {video_id},
    "video_title" : {video_title},
    "chunk_text": "Text of the logical chunk here...",
    "chunk_title": "Concise descriptive title here...",
    "keywords": ["keyword1", "keyword2", "keyword3", ...]
  }},
  ...
]

Here is the input chunks (each chunk is already a continuous string of transcript lines):

{single_chunk}

Remember: The goal is to create **logical, semantically meaningful chunks** with metadata (`title` and `keywords`) that can later be stored in a vector database or used for retrieval.
Output must be strictly valid JSON. Do not include any text outside the JSON.
Use double quotes for keys and strings.
Do NOT wrap the JSON in code blocks, backticks, or any other formatting.
Do not include any text outside the JSON.
"""
id = 0
all_chunks = []
for chunk in text_chunks:
  values = {
    "single_chunk" :   chunk,
    "video_id" : video_id,
    "video_title" : video_title
  }
  prompt = llm_chunking_prompt_template.format(**values).strip()
  try:
    response = json5.loads(llm(prompt))
  except Exception as e:
    print("Error parsing LLM Output:", e)
    response = []
  for r in response:
    r["chunk_id"] = f"{video_id}_{id}"
    id += 1
  all_chunks.extend(response)

In [55]:
len(all_chunks)

84

In [56]:
all_chunks[1]

{'video_id': 'y3yAVZk3tyA',
 'video_title': "Keyu Jin: China's Economy, Tariffs, Trade, Trump, Communism & Capitalism | Lex Fridman Podcast #477",
 'chunk_text': "What is the single biggest misconception the West has about China's economy today? - The biggest misunderstanding is somehow that a group of people, or even just one person, runs the entire Chinese economy. It is far from the reality. It is a very complex, large economy. And even if there is an extreme form of political centralization, the economy is totally decentralized. The role that the local mayors, I call this the mayor economy, plays in reforms, but also driving the technological innovation that we're seeing right now, it is actually not run by just a handful of people. It's more decentralized than the US's. And I think more broadly, a big misunderstanding is really the relationship between Chinese people and authority.",
 'chunk_title': "Misconceptions about China's Economic Structure",
 'keywords': ['misconception',


In [57]:
type(all_chunks)

list

In [58]:
with open("../data/llm_chunks.json", "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)